In [1]:
import requests 
import pandas as pd
import re
from glob import glob
from bs4 import BeautifulSoup

In [5]:
response = requests.get('https://d18rn0p25nwr6d.cloudfront.net/CIK-0000320193/4027b00f-7593-41b2-8996-2f68839e44ae.html')
soup = BeautifulSoup(response.content)

In [6]:
soup.prettify

<bound method Tag.prettify of <!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 5.0 Transitional//EN">
<html>
<body><!-- 

APPLE INC. - Company Name
7/29/2022 - Filing date
Quarterly Report - Filing description
10-Q - Filing Form

-->
<style>
.q4Style
{
	font-family: 'Arial',Arial,sans-serif;
	font-size: 12px;
}
.ModuleFilingTitle
{
font-size: 15px;
font-weight: bold;
}
.ModuleFilingDescription
{
	margin: 10px 0 10px 0;
}
</style>
<div class="Q4SECFiling q4Style">
<div class="ModuleFilingTitle">
<span>APPLE INC., 10-Q filed on 7/29/2022</span>
</div>
<div class="ModuleFilingDescription">
<span>Quarterly Report</span>
</div>
</div>
<link href="https://content.edgar-online.com/rxbrlviewer/css/OrgImetrixStyle.css" rel="Stylesheet" type="text/css"/>
<link href="https://content.edgar-online.com/rxbrlviewer/css/jquery-ui-1.8.1.custom.css" rel="stylesheet" type="text/css"/>
<link href="https://content.edgar-online.com/rxbrlviewer/css/dropdownbuttons.css" rel="stylesheet" type="text/css"/>
<link href="h

In [8]:
#re.search(r'<span class="review-count rating-qualifier">\n\s+(\d*)',soup.contents).group(1)
# Look for these:
lookforthese = '''
<div class="megamenu" id="statementsMenu">

 <li><a class="reportLink markFirstAsSelected" href="#" reportid="Report2">CONDENSED CONSOLIDATED STATEMENTS OF OPERATIONS (Unaudited)</a></li>
 <li><a class="reportLink" href="#" reportid="Report3">CONDENSED CONSOLIDATED STATEMENTS OF COMPREHENSIVE INCOME (Unaudited)</a></li>
 <li><a class="reportLink" href="#" reportid="Report4">CONDENSED CONSOLIDATED BALANCE SHEETS (Unaudited)</a></li>
 <li><a class="reportLink" href="#" reportid="Report5">CONDENSED CONSOLIDATED BALANCE SHEETS (Unaudited) (Parenthetical)</a></li>
 <li><a class="reportLink" href="#" reportid="Report6">CONDENSED CONSOLIDATED STATEMENTS OF SHAREHOLDERS' EQUITY (Unaudited)</a></li>
 <li><a class="reportLink" href="#" reportid="Report7">CONDENSED CONSOLIDATED STATEMENTS OF CASH FLOWS (Unaudited)</a></li>

'''

In [11]:
table = soup.findAll('table', class_ = 'report')[6]
out = []
for row in table.find_all('tr'):
    tl = row.text.split('\n')
    out.append(tl)
rowlist = []
for x in out:
    rowlist.append(list(filter(None, x)))

print(rowlist[:3])
colnames_dates = rowlist[1] # Entries with dates
rowlist = rowlist[3:] # 
colnames_dates

[['CONDENSED CONSOLIDATED STATEMENTS OF CASH FLOWS (Unaudited) - USD ($) $ in Millions', '9 Months Ended'], ['Jun. 25, 2022', 'Jun. 26, 2021'], ['Statement of Cash Flows [Abstract]', '\xa0', '\xa0']]


['Jun. 25, 2022', 'Jun. 26, 2021']

In [12]:
colnames = ['ID(Millions)']+ colnames_dates # ID + Dates that were in the DF 
cashflows = pd.DataFrame(columns=colnames)

#Iterate over the list, appending to the dataframe
for i in range(0, len(rowlist)):
    cashflows.loc[i, colnames[0]] = rowlist[i][0]
    cashflows.loc[i, colnames[1]] = rowlist[i][1]
    cashflows.loc[i, colnames[2]] = rowlist[i][2]
# Remove the \xa0 character, replace with NaN
cashflows = cashflows.replace('\xa0', float('NaN')).dropna()
# Finally, Clean the data entries by taking out necessary chars
remove_these_char = ['\$', ',']
replace_char_with = ['','']
coldict = dict(zip(remove_these_char, replace_char_with))


In [13]:
#cashflows.T.replace(coldict, regex=True).replace( '[(]','-',regex=True).replace( '[)]','',regex=True).T

In [14]:
def gather_cashflows(link):
    # Use link to get CF statements
    response = requests.get(link)
    response.close()
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.findAll('table', class_ = 'report')[6]
    out = []
    for row in table.find_all('tr'):
        tl = row.text.split('\n')
        out.append(tl)
    rowlist = []
    for x in out:
        rowlist.append(list(filter(None, x)))

    print(rowlist[:3])
    colnames_dates = rowlist[1] # Entries with dates
    rowlist = rowlist[3:] # 
    colnames = ['ID(Millions)']+ colnames_dates # ID + Dates that were in the DF 
    cashflows = pd.DataFrame(columns=colnames)

    #Iterate over the list, appending to the dataframe
    for i in range(0, len(rowlist)):
        cashflows.loc[i, colnames[0]] = rowlist[i][0]
        cashflows.loc[i, colnames[1]] = rowlist[i][1]
        cashflows.loc[i, colnames[2]] = rowlist[i][2]
    # Remove the \xa0 character, replace with NaN
    cashflows = cashflows.replace('\xa0', float('NaN')).dropna()
    # Finally, Clean the data entries by taking out necessary chars
    remove_these_char = ['\$', ',']
    replace_char_with = ['','']
    coldict = dict(zip(remove_these_char, replace_char_with))

    return cashflows[colnames[:2]].T.replace(coldict, regex=True).replace( '[(]','-',regex=True).replace( '[)]','',regex=True).T

In [4]:
#fnames = [x for x in glob('earnings_excel/*')]
#df = pd.read_excel(fnames[0],header=16,index_col=1,sheet_name='BALANCE_SHEET', engine = 'xlrd').T

linklist=['https://d18rn0p25nwr6d.cloudfront.net/CIK-0000320193/4027b00f-7593-41b2-8996-2f68839e44ae.html',
        'https://d18rn0p25nwr6d.cloudfront.net/CIK-0000320193/9765233e-06ae-4e3f-a7f3-cede782b3422.html', # April 
        'https://d18rn0p25nwr6d.cloudfront.net/CIK-0000320193/3fd37602-388a-4a6d-9be4-7fe62828ee97.html', # Jan
        'https://d18rn0p25nwr6d.cloudfront.net/CIK-0000320193/4aa1bcd3-a930-4530-a31c-889454745f49.html',
        'https://d18rn0p25nwr6d.cloudfront.net/CIK-0000320193/6addfc26-883c-4600-9b48-23d27b360c73.html',
        'https://d18rn0p25nwr6d.cloudfront.net/CIK-0000320193/6d1bb203-09d8-4c92-bc59-cc6a60be87a4.html',
        'https://d18rn0p25nwr6d.cloudfront.net/CIK-0000320193/0cbfcdf8-bddb-4922-8b3d-f82b2c98e842.html',
        'https://d18rn0p25nwr6d.cloudfront.net/CIK-0000320193/eaf915d2-5f0d-415d-bc4f-b801c509b3ac.html',
        'https://d18rn0p25nwr6d.cloudfront.net/CIK-0000320193/4eed2721-55f6-4f14-8726-417ba134a96d.html',
        'https://d18rn0p25nwr6d.cloudfront.net/CIK-0000320193/ac5b00f9-0058-45cc-9f2f-bb08416eaa73.html']

lodf = [gather_cashflows(x) for x in linklist]

[['CONDENSED CONSOLIDATED STATEMENTS OF CASH FLOWS (Unaudited) - USD ($) $ in Millions', '9 Months Ended'], ['Jun. 25, 2022', 'Jun. 26, 2021'], ['Statement of Cash Flows [Abstract]', '\xa0', '\xa0']]
[['CONDENSED CONSOLIDATED STATEMENTS OF CASH FLOWS (Unaudited) - USD ($) $ in Millions', '6 Months Ended'], ['Mar. 26, 2022', 'Mar. 27, 2021'], ['Statement of Cash Flows [Abstract]', '\xa0', '\xa0']]
[['CONDENSED CONSOLIDATED STATEMENTS OF CASH FLOWS (Unaudited) - USD ($) $ in Millions', '3 Months Ended'], ['Dec. 25, 2021', 'Dec. 26, 2020'], ['Statement of Cash Flows [Abstract]', '\xa0', '\xa0']]
[['CONDENSED CONSOLIDATED STATEMENTS OF CASH FLOWS (Unaudited) - USD ($) $ in Millions', '9 Months Ended'], ['Jun. 26, 2021', 'Jun. 27, 2020'], ['Statement of Cash Flows [Abstract]', '\xa0', '\xa0']]
[['CONDENSED CONSOLIDATED STATEMENTS OF CASH FLOWS (Unaudited) - USD ($) $ in Millions', '6 Months Ended'], ['Mar. 27, 2021', 'Mar. 28, 2020'], ['Statement of Cash Flows [Abstract]', '\xa0', '\xa0']]


In [15]:
from functools import reduce 
import datetime as dt 
alldf = reduce(lambda x, y : pd.merge(x,y), lodf).T
alldf.columns = alldf.iloc[0]
alldf = alldf.iloc[1:]
alldf.index = [dt.datetime.strptime(x,'%b. %d, %Y') for x in alldf.index.to_list()]
alldf.index.name = 'Date'
alldf

ID(Millions),Cash cash equivalents and restricted cash beginning balances,Net income,Depreciation and amortization,Share-based compensation expense,Other,Other,Other,Other,Other,Other,...,Proceeds from sales of marketable securities,Payments for acquisition of property plant and equipment,Payments for taxes related to net share settlement of equity awards,Payments for dividends and dividend equivalents,Repurchases of common stock,Repayments of term debt,Cash used in financing activities,Cash cash equivalents and restricted cash ending balances,Cash paid for income taxes net,Cash paid for interest
Date,,,,,,,,,,,,,,,,,,,,,
2022-06-25,35929,79082,8239,6760,-61,-61,-61,-61,-61,-61,...,33609,-7419,-5915,-11138,-64974,-6750,-83955,28861,12251,1910
2022-03-26,35929,59640,5434,4517,-20,-20,-20,-20,-20,-20,...,24668,-5317,-3218,-7327,-43109,-3750,-56510,29180,9301,1406
2021-12-25,35929,34630,2697,2265,167,167,167,167,167,167,...,10675,-2803,-2888,-3732,-20478,0,-28159,38630,5235,531
2021-06-26,39789,74129,8295,5961,-689,-689,-689,-689,-689,-689,...,36745,-7862,-5855,-10827,-66223,-7500,-72971,35276,18536,1870
2021-03-27,39789,52385,5463,4001,-474,-474,-474,-474,-474,-474,...,21645,-5769,-3160,-7060,-43323,-4500,-43575,40006,10276,1327
2020-12-26,39789,28755,2666,2020,25,25,25,25,25,25,...,9344,-3500,-2861,-3613,-24775,-1000,-32249,37719,1787,619
2020-06-27,50224,44738,8354,5105,-94,-94,-94,-94,-94,-94,...,39760,-5525,-3234,-10570,-55171,-12629,-65463,35039,8410,2275
2020-03-28,50224,33485,5602,3407,-259,-259,-259,-259,-259,-259,...,27762,-3960,-1566,-6914,-39280,-5250,-46347,43049,7505,1689
2019-12-28,50224,22236,2816,1710,-142,-142,-142,-130,-130,-130,...,7280,-2107,-1379,-3539,-20706,-1000,-25407,41665,4393,771


In [375]:
alldf.to_csv('aapl_cashflow_to_2018.csv')


In [331]:
import datetime as dt 

In [342]:
ind = alldf.index.to_list()[0]

In [365]:
alldf

ID(Millions),Cash cash equivalents and restricted cash beginning balances,Net income,Depreciation and amortization,Share-based compensation expense,Other,Other,Other,Other,Other,Other,...,Proceeds from sales of marketable securities,Payments for acquisition of property plant and equipment,Payments for taxes related to net share settlement of equity awards,Payments for dividends and dividend equivalents,Repurchases of common stock,Repayments of term debt,Cash used in financing activities,Cash cash equivalents and restricted cash ending balances,Cash paid for income taxes net,Cash paid for interest
Date,,,,,,,,,,,,,,,,,,,,,
